### Homework 5: Question search engine

Remeber week01 where you used GloVe embeddings to find related questions? That was.. cute, but far from state of the art. It's time to really solve this task using context-aware embeddings.

__Warning:__ this task assumes you have seen `seminar.ipynb`!

In [ ]:
%pip install --upgrade transformers datasets accelerate deepspeed
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.0 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.12.6-py3-none-any.whl size=1306735 sha256=9c18e726bd0ca2d61cab4b4e982970c4567390a0c6b2cb05300b685d4af9681e
  Stored in directory: /root/.cache/pip/wheels/a3/dc/a2/f585faaed4dec8

### Load data and model

In [ ]:
qqp = datasets.load_dataset('SetFit/qqp')
print('\n')
print("Sample[0]:", qqp['train'][0])
print("Sample[3]:", qqp['train'][3])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]



Sample[0]: {'text1': 'How is the life of a math student? Could you describe your own experiences?', 'text2': 'Which level of prepration is enough for the exam jlpt5?', 'label': 0, 'idx': 0, 'label_text': 'not duplicate'}
Sample[3]: {'text1': 'What can one do after MBBS?', 'text2': 'What do i do after my MBBS ?', 'label': 1, 'idx': 3, 'label_text': 'duplicate'}


In [ ]:
model_name = "gchhablani/bert-base-cased-finetuned-qqp"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

### Tokenize the data

In [ ]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [ ]:
qqp_preprocessed

DatasetDict({
    train: Dataset({
        features: ['text1', 'text2', 'label', 'idx', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 363846
    })
    validation: Dataset({
        features: ['text1', 'text2', 'label', 'idx', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 40430
    })
    test: Dataset({
        features: ['text1', 'text2', 'label', 'idx', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 390965
    })
})

In [ ]:
print(str(qqp_preprocessed['train'][0]['input_ids'])[:100], "...")

[101, 1731, 1110, 1103, 1297, 1104, 170, 12523, 2377, 136, 7426, 1128, 5594, 1240, 1319, 5758, 136,  ...


### Task 1: evaluation (1 point)

We randomly chose a model trained on QQP - but is it any good?

One way to measure this is with validation accuracy - which is what you will implement next.

Here's the interface to help you do that:

__Your task__ is to measure the validation accuracy of your model.
Doing so naively may take several hours. Please make sure you use the following optimizations:

- run the model on GPU with no_grad
- using batch size larger than 1
- use optimize data loader with num_workers > 1
- (optional) use [mixed precision](https://pytorch.org/docs/stable/notes/amp_examples.html)


In [ ]:
from tqdm import tqdm
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=256, shuffle=False, collate_fn=transformers.default_data_collator,num_workers = 2
)
model = model.to('cuda')
correct = 0
with torch.no_grad():
    for batch in tqdm(val_loader):
        y = batch['labels']
        predicted = model(
        input_ids=batch['input_ids'].to('cuda'),
        attention_mask=batch['attention_mask'].to('cuda'),
        token_type_ids=batch['token_type_ids'].to('cuda')
        )
        preds = torch.softmax(predicted.logits, dim=1).data.cpu().numpy()
        correct += torch.sum(torch.tensor(preds.argmax(axis = -1)) == y,dtype=torch.float32).item()

100%|██████████| 158/158 [04:43<00:00,  1.79s/it]


In [ ]:
accuracy = correct / val_set.num_rows
accuracy

0.9083848627256987

In [ ]:
assert 0.9 < accuracy < 0.91

### Task 2: train the model (4 points)

For this task, you have two options:

__Option A:__ fine-tune your own model. You are free to choose any model __except for the original BERT.__ We recommend [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base). Better yet, choose the best model based on public benchmarks (e.g. [GLUE](https://gluebenchmark.com/)).

You can write the training code manually or use transformers.Trainer (see [this example](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification)). Please make sure that your model's accuracy is at least __comparable__ with the above example for BERT.


__Option B:__ compare at least 3 pre-finetuned models (in addition to the above BERT model). For each model, report (1) its accuracy, (2) its speed, measured in samples per second in your hardware setup and (3) its size in megabytes. Please take care to compare models in equal setting, e.g. same CPU / GPU. Compile your results into a table and write a short (~half-page on top of a table) report, summarizing your findings.

### До тьюна

In [ ]:
model_name = "roberta-base"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)
model = model.to('cuda')

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [ ]:
qqp_preprocessed

DatasetDict({
    train: Dataset({
        features: ['text1', 'text2', 'label', 'idx', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 363846
    })
    validation: Dataset({
        features: ['text1', 'text2', 'label', 'idx', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 40430
    })
    test: Dataset({
        features: ['text1', 'text2', 'label', 'idx', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 390965
    })
})

In [ ]:
from tqdm import tqdm
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=256, shuffle=False, collate_fn=transformers.default_data_collator,num_workers = 2
)
model = model.to('cuda')
correct = 0
with torch.no_grad():
    for batch in tqdm(val_loader):
        y = batch['labels']
        predicted = model(
        input_ids=batch['input_ids'].to('cuda'),
        attention_mask=batch['attention_mask'].to('cuda'),
        )
        preds = torch.softmax(predicted.logits, dim=1).data.cpu().numpy()
        correct += torch.sum(torch.tensor(preds.argmax(axis = -1)) == y,dtype=torch.float32).item()

100%|██████████| 158/158 [04:05<00:00,  1.55s/it]


In [ ]:
accuracy = correct / val_set.num_rows
accuracy

0.6318327974276527

### Тьюн + результаты после

In [ ]:
args = transformers.TrainingArguments(
    output_dir="./finetuned_model",
    num_train_epochs = 5,
    per_device_train_batch_size=128,
)
trainer = transformers.Trainer(
    model = model,
    args = args,
    train_dataset = qqp_preprocessed['train'],
    eval_dataset = qqp_preprocessed['validation']
)
trainer.train()

Step,Training Loss
500,0.366500
1000,0.306400
1500,0.286200
2000,0.272900
2500,0.265200
3000,0.245900
3500,0.209100
4000,0.207800
4500,0.206300


In [ ]:
from tqdm import tqdm
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=256, shuffle=False, collate_fn=transformers.default_data_collator,num_workers = 2
)
model = model.to('cuda')
correct = 0
with torch.no_grad():
    for batch in tqdm(val_loader):
        y = batch['labels']
        predicted = model(
        input_ids=batch['input_ids'].to('cuda'),
        attention_mask=batch['attention_mask'].to('cuda'),
        #token_type_ids=batch['token_type_ids'].to('cuda')
        )
        preds = torch.softmax(predicted.logits, dim=1).data.cpu().numpy()
        correct += torch.sum(torch.tensor(preds.argmax(axis = -1)) == y,dtype=torch.float32).item()

100%|██████████| 158/158 [04:46<00:00,  1.81s/it]


In [ ]:
accuracy = correct / val_set.num_rows
accuracy

0.8935938659411328

### Task 3: try the full pipeline (1 point)

Finally, it is time to use your model to find duplicate questions.
Please implement a function that takes a question and finds top-5 potential duplicates in the training set. For now, it is fine if your function is slow, as long as it yields correct results.

Showcase how your function works with at least 5 examples.

### Загрузка модели и проверка accuracy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

zip_file_path = '/content/drive/MyDrive/finetuned_model.zip'
extract_folder = '/home'

os.makedirs(extract_folder, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

In [ ]:
checkpoint_path = '/home/checkpoint-4000/'

model = transformers.RobertaForSequenceClassification.from_pretrained(checkpoint_path)
model_name = "roberta-base"

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [ ]:
from tqdm import tqdm
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=256, shuffle=False, collate_fn=transformers.default_data_collator,num_workers = 2
)
model = model.to('cuda')
correct = 0
with torch.no_grad():
    for batch in tqdm(val_loader):
        y = batch['labels']
        predicted = model(
        input_ids=batch['input_ids'].to('cuda'),
        attention_mask=batch['attention_mask'].to('cuda'),
        )
        preds = torch.softmax(predicted.logits, dim=1).data.cpu().numpy()
        correct += torch.sum(torch.tensor(preds.argmax(axis = -1)) == y,dtype=torch.float32).item()

100%|██████████| 158/158 [04:24<00:00,  1.67s/it]


In [ ]:
accuracy = correct / val_set.num_rows
accuracy

0.9011377689834281

### Поиск совпадений

In [ ]:
def find_nearest(similar_to_string,qqp,tokenizer,model):
    MAX_LENGTH = 128

    def preprocess_function(examples):
        result = tokenizer(
            examples['text1'], [similar_to_string] * len(examples['text1']),
            padding='max_length', max_length=MAX_LENGTH, truncation=True
        )
        result['label'] = examples['label']
        return result

    train_set = qqp['train'].map(preprocess_function, batched=True)

    texts = train_set['text1']

    nearest = {}

    train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=1024, shuffle=False, collate_fn=transformers.default_data_collator,num_workers = 2
    )

    with torch.no_grad():
        for i,batch in tqdm(enumerate(train_loader)):
            predicted = model(
            input_ids=batch['input_ids'].to('cuda'),
            attention_mask=batch['attention_mask'].to('cuda'),
            )
            predicted_equal = torch.softmax(predicted.logits, dim=1).data.cpu().numpy()
            for idx in range(len(predicted_equal)):
                nearest[texts[batch['idx'][idx]]] = predicted_equal[idx][1] # in 0 - probability of not duplicate, in 1 - duplicate

    sorted_nearest = sorted(nearest.items(), key=lambda x: x[1], reverse=True)

    for text, similarity in sorted_nearest[:10]:
        print(f"Текст: {text}, Близость: {similarity}")



In [ ]:
find_nearest('Do you like Machine Learning?',qqp,tokenizer,model)

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

356it [38:07,  6.42s/it]


Текст: Is there any proof of the existence of aliens? Has anyone seen aliens?, Близость: 0.922537624835968
Текст: I want to do a survey about online digital marketing courses. How many of you are experienced in online digital marketing courses? What is your opinion on it?, Близость: 0.7945742011070251
Текст: What is outside our universe? Also does it make sense to ask what is outside of the outside of universe?, Близость: 0.7448331117630005
Текст: What is nano fuel? What is the use of nano fuel?, Близость: 0.6857314705848694
Текст: I am from non programming background, how can I learn machine learning by self?, Близость: 0.5962274074554443
Текст: What is PSD to HTML conversion? What is the use of PSD to HTML conversion?, Близость: 0.5848731994628906
Текст: How does a total beginner start to learn machine learning?, Близость: 0.47568902373313904
Текст: I have know about bluetooth module? What are other prerequisite thing should I know to start bluetooth module?, Близость: 0.449200153350